# Tutorial
_This tutorial notebook can be downloaded and ran on your own system._

In [ ]:
from pimf import PowerLawIMF

import numpy as np
import matplotlib.pyplot as plt

Let's start by looking at the doctring:

In [ ]:
help(PowerLawIMF)

It's hard to read the unformatted latex (so the [readthedocs](https://pimf.readthedocs.io/en/latest/apidocs/pimf/pimf.initialmassfunction.html#pimf.initialmassfunction.PowerLawIMF) might be better) but we can see that this class implements an IMF that looks like $$\xi(m)dm = \xi_0 m^\alpha dm,$$ with a default value of $\alpha=-2.35$, to match [Salpeter (1955)](https://ui.adsabs.harvard.edu/abs/1955ApJ...121..161S/abstract).

## Creating an IMF instance
In some cases it's as simple as:

In [ ]:
my_imf = PowerLawIMF()

because we have tried to choose sensible defaults. In other IMFs, there aren't necessarily sensible defaults. The parameters you are most likely to want to tweak are probably `normalisation` and `normalisation_value`, which we explain more [below](#normalisation).

## Generating Values
All IMFs have a `__call__` method, so can be called like functions in python. This just evaluates $\xi(M)$. You can pass floats or numpy arrays.

In [ ]:
print(my_imf(1))
print(my_imf(10))

In [ ]:
# Generate mass values
M = np.geomspace(0.1, 100)
print(my_imf(M))

In [ ]:
plt.loglog(M, my_imf(M))
plt.xlabel("Mass ($M_\\odot$)")
plt.ylabel("Number of Stars (arbitrary units)")
plt.show()

## Integrating
Let's say we wanted to calculate the number of black holes that form. A simple model might be that any stars below 25 solar masses don't form a black hole, while stars above this limit do. We could use something like the [trapezium rule](https://en.wikipedia.org/wiki/Trapezoidal_rule) or a [numerical integration routine](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html) with the `__call__` method. But! We can do better.

For most IMFs it is easy enough to write down an analytic solution to the integral, and that is what we implement in the `integrate` method.

In [ ]:
number_stars_below_25Msol = my_imf.integrate(0.1, 25)
number_stars_above_25Msol = my_imf.integrate(25, 100)
print(f"There are {number_stars_below_25Msol} stars below 25 solar masses, and {number_stars_above_25Msol} above.")

At this point, you _should_ be wondering what these values actually mean. After all, you never said how many stars there were in total, or the total stellar mass (which is more common). That is what we will cover in [normalisation](#normalisation).

## Normalisation
We provide a set of methods for normalising the IMF, but it's easiest to do it when you initialise the class using the `normalise` and `normalisation_value` arguments. When you print an imf, it will tell you how its been normalised.

These fix the value of $\xi_0$ by calculating the integral, or taking a value from the user.

### normalise
There are three options for `normalise`:
1. "mass", a string, requires `normalisation_value`
2. "number", a string, requires `normalisation_value`
3. Any int or float

#### "mass"
When normalising by mass we rearrange $$\int^{M_\mathrm{max}}_{M_\mathrm{min}}M\xi(M)\mathrm{d}M = \texttt{normalisation_value}$$

#### "number"
When normalising by number we rearrange $$\int^{M_\mathrm{max}}_{M_\mathrm{min}}\xi(M)\mathrm{d}M = \texttt{normalisation_value}$$

#### Any int or float
If you pass a number as opposed to one of the strings, then it just uses this value for $\xi_0$.

Let's say we want to create an IMF that represents 2000 stars, and ask how many black holes that forms.

In [ ]:
two_thousand_stars = PowerLawIMF(normalisation="number", normalisation_value=2000)
number_stars_below_25Msol = two_thousand_stars.integrate(0.1, 25)
number_stars_above_25Msol = two_thousand_stars.integrate(25, 100)
print(f"There are {number_stars_below_25Msol} stars below 25 solar masses, and {number_stars_above_25Msol} above.")

Or, we could ask how many sun-like stars are there in the milky way? Taking a milky way mass of [$\sim6\times10^{10}M_\odot$](https://arxiv.org/abs/1407.1078)

In [ ]:
milky_way = PowerLawIMF(normalisation="mass", normalisation_value=6e10)
print(f"There are {milky_way(1):g} solar mass stars in the milky way, and {milky_way.integrate(0.9, 1.1):g} within 10% of that.")
### !!! Why is the first one higher?

Winner of the longest method name goes to...
## `integrate_linear_piecewise_interpolated_product`
This is not something that everyone will have to use, but it's code I wrote so I might as well include it. Essentially what this does is solve annoying problems when you want to know the integral of an arbitrary function $$\int^{M_\textrm{max}}_{M_\textrm{min}} f(M)\xi(M)dM,$$ but $f(M)$ is only defined on a grid. For example, if someone had ran a bunch of stellar models to compute luminosity.

In [ ]:
# We take some data from the GENEC grids: https://obswww.unige.ch/Research/evol/tables_grids2011/
# Everything in solar units
grid_masses = np.array([0.8, 0.9, 1.0, 1.1, 1.25, 1.35, 1.5, 1.7, 2.0, 2.5, 3.0, 4.0, 5.0, 7, 9, 12, 15, 20, 25, 32, 40, 60, 85, 120])
grid_luminosities = 10**np.array([-0.58999, -0.35509, -0.13714, 0.06467, 0.29537, 0.459878, 0.673904, 0.913744, 1.208899, 1.596045, 1.898976, 2.365394, 2.712744, 3.220316, 3.584321, 3.977491, 4.266387, 4.614872, 4.86599, 5.125097, 5.343182, 5.703098, 5.980312, 6.231065])

plt.loglog(grid_masses, grid_luminosities, "o")
plt.xlabel("Mass / $M_\\odot$")
plt.ylabel("Luminosity / $L_\\odot$")
plt.show()

Above we have our grid of mass and luminosity. Let's use it to answer the question:

_Do massive stars contribute more or less to luminosity than low mass stars?_

We can see above that luminosity increases increases with mass, but we know that more massive stars are less common. Which effect wins?

In [ ]:
# This will return the total luminosity of stars between 0.8 and 8 solar masses, in solar luminosities
low_mass_contribution = my_imf.integrate_linear_piecewise_interpolated_product(
    0.8,               # Minimum mass for integration bounds
    8,                 # Maximum mass for integration bounds
    grid_masses,       # Masses we are interpolating
    grid_luminosities  # Luminosities we are interpolating, the unit of these is our final unit
)

# This will return the total luminosity of stars between 8 and 100 solar masses, in solar luminosities.
high_mass_contribution = my_imf.integrate_linear_piecewise_interpolated_product(
    8,                 # Minimum mass for integration bounds
    100,               # Maximum mass for integration bounds
    grid_masses,       # Masses we are interpolating
    grid_luminosities  # Luminosities we are interpolating, the unit of these is our final unit
)

print(f"{100*low_mass_contribution / (low_mass_contribution + high_mass_contribution) : .2f}% of luminosity comes from low-mass stars")
print(f"{100*high_mass_contribution / (low_mass_contribution + high_mass_contribution) :.2f}% of luminosity comes from high-mass stars")

So massive stars hugely dominate!